In [4]:
import pandas as pd
import json

In [5]:

ruta='pisos_limpios.csv'
dfknn = pd.read_csv(ruta,sep=',')
dfknn.columns

Index(['uv', 'estado', 'precio', 'long', 'lat', 'provincia', 'ciudad',
       'distrito', 'planta', 'habitaciones', 'banos', 'metros',
       'fecha_construccion', 'intext', 'portero', 'ascensor',
       'parque_infantil', 'terraza', 'trastero', 'cuarto_de_basura',
       'zonas_comunes', 'piscina', 'garaje', 'calefaccion', 'jardin',
       'aire_acondicionado', 'cusec', 'precio_area', 'reforma',
       'precio_reformado', 'ns', 'eo'],
      dtype='object')

In [7]:
with open('pesos.json') as archivo:
            opciones = json.load(archivo)

In [8]:
filtros=opciones['filtros']


In [5]:
dfknn.columns

Index(['uv', 'estado', 'precio', 'long', 'lat', 'provincia', 'ciudad',
       'distrito', 'planta', 'habitaciones', 'banos', 'metros',
       'fecha_construccion', 'intext', 'portero', 'ascensor',
       'parque_infantil', 'terraza', 'trastero', 'cuarto_de_basura',
       'zonas_comunes', 'piscina', 'garaje', 'calefaccion', 'jardin',
       'aire_acondicionado', 'cusec', 'precio_area', 'reforma',
       'precio_reformado', 'ns', 'eo'],
      dtype='object')

In [6]:
filtros=filtros[7:9]
filtros

['banos', 'portero']

In [7]:
df = dfknn
target = df.iloc[1]
df.drop(1, axis=0, inplace=True)

for f in filtros:
    index = df[ df[f] != target[f]  ].index
    df.drop(index, axis=0, inplace=True)
    df.drop(f,axis=1, inplace=True)
    





ValueError: No axis named row for object type Series

KeyError: "None of [Index(['banos', 'portero'], dtype='object')] are in the [columns]"

In [9]:


        index = df[ df[f] != target[f]  ].index
        df.drop(index, axis=0, inplace=True)
        
        


In [10]:
df

,uv,estado,precio,long,lat,provincia,ciudad,distrito,planta,habitaciones,...,garaje,calefaccion,jardin,aire_acondicionado,cusec,precio_area,reforma,precio_reformado,ns,eo
1,UV/2020/000004,1,125.0,40.1924,-3.66922,1,Valdemoro,Zona estación,1,2,...,0,0,0,0,2.816101e+09,NaN,35,160.0,S,E
2,UV/2020/000006,1,0.0,40.4330,-3.63715,1,Madrid,Ciudad Lineal,2,2,...,0,0,0,0,2.807915e+09,136690.000000,35,35.0,S,NaN
3,UV/2020/000007,1,159.0,40.2950,-3.80339,1,Fuenlabrada,La Serna,9,3,...,0,0,0,0,2.805806e+09,NaN,35,194.0,S,E
4,UV/2020/000008,1,170.0,40.2846,-3.78862,1,Fuenlabrada,Centro,6,3,...,0,0,0,0,2.805802e+09,NaN,45,215.0,S,E
5,UV/2020/000009,1,170.0,40.2839,-3.78976,1,Fuenlabrada,Centro,6,3,...,0,0,0,0,2.805802e+09,NaN,35,205.0,N,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,UV/2021/000378,1,225.0,41.3735,2.17075,2,Barcelona,Sants-Montjuïc,7,2,...,0,0,0,0,8.019030e+08,272568.421053,35,260.0,S,E
159,UV/2021/000428,1,125.0,40.3557,-3.69495,1,Madrid,Villaverde,4,3,...,0,0,0,0,2.807917e+09,149666.666667,45,170.0,S,NaN
161,UV/2021/000430,2,239.0,41.3745,2.12767,2,Barcelona,Sants-Montjuïc,2,2,...,0,0,0,0,8.019031e+08,236071.428571,35,274.0,S,E
162,UV/2021/000433,1,300.0,41.3809,2.17924,2,Barcelona,Ciutat Vella,0,4,...,0,0,0,0,8.019010e+08,477259.797980,55,355.0,S,NaN
